In [1]:
# allows to import of modules
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
nb_dir
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
from pathlib import Path
import time

import pandas as pd
import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision.models as models

import metrics
import model
import datatset
import utils
from eval import Eval

In [3]:
DATA_DIR = Path("/home/shaun/personal/kaggle-data/nybg2020/")


In [4]:
checkpoints_dir = DATA_DIR/"models-1/model-114720.pth"

In [5]:
# load and update labels
df = pd.read_csv(DATA_DIR/"train/train.csv")
df = df[["file_name", "category_id"]]
classes_map = utils.load_json(DATA_DIR/"classes-map.json")

In [6]:
# Update class labels
new_labels = [classes_map[str(i)] for i in df["category_id"]]
df["new_labels"] = new_labels

In [7]:
ds = datatset.HerbDataSet(df, DATA_DIR/"train", 256, label="new_labels")
loader = DataLoader(ds, batch_size=16, shuffle=True)

In [8]:
# Metrics
acc = metrics.Accuracy()
f1 = metrics.F1()
writer = SummaryWriter(DATA_DIR/"validation-logs")
manager = metrics.MetricManager([acc, f1], writer)

# Loss
criterion = nn.CrossEntropyLoss()

In [9]:
# Model
m = model.get_model(models.resnet50(pretrained=False), p1=0, p2=0)

In [10]:
Eval(manager, criterion, loader, m, checkpoints_dir).run_once()

/home/shaun/personal/.pytorch-env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/shaun/personal/.pytorch-env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


KeyboardInterrupt: 